<a href="https://colab.research.google.com/github/PaulMercerAI/AML19/blob/master/IntroductoryNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl https://raw.githubusercontent.com/PaulMercerAI/AML19/master/data/20191111_Transactions.csv.xz -L | xzcat > transactions.csv 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5697k  100 5697k    0     0  3134k      0  0:00:01  0:00:01 --:--:-- 3132k


In [3]:
!curl https://raw.githubusercontent.com/PaulMercerAI/AML19/master/data/20191111_Accounts.csv.xz -L | xzcat > accounts.csv 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  303k  100  303k    0     0  1273k      0 --:--:-- --:--:-- --:--:-- 1273k


In [4]:
#!head transactions.csv
!pip install node2vec

In [0]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import node2vec

In [0]:
transactions = pd.read_csv('transactions.csv')
accounts = pd.read_csv('accounts.csv')

In [16]:
#transactions[transactions['is_suspx']==True].tail(10)
transactions.head(10)

,id,timestamp,sender_account,receiver_account,amount,is_suspx,alert_id,alert_type
0,1,2017-01-01T00:00:00Z,6f339c5e-0392-4cfa-9d9f-2c8c6640464a,50cca99f-5ed2-465d-8c07-cdc393bc506c,397.89,Unknown,Unknown,Unknown
1,2,2017-01-01T00:00:00Z,a117b864-82f5-4390-ad69-6cb3b774b61d,f46a9817-194a-489d-8f3f-be079dba68ea,319.16,False,-,-
2,3,2017-01-01T00:00:00Z,2e0c881c-1068-4337-ae5e-ccf7df4924f2,39681e1d-898a-4752-9c71-64a818906db4,262.26,Unknown,Unknown,Unknown
3,4,2017-01-01T00:00:00Z,de72d970-10c4-4f9e-b020-321be90f4137,5eabfb1c-fda3-4293-97b3-0cb4bcf71f08,288.52,False,-,-
4,5,2017-01-01T00:00:00Z,68a522b9-ff6f-46ca-a245-932771224f95,42bb8f63-4145-4abc-8cc2-e0f013093c17,344.55,Unknown,Unknown,Unknown
5,6,2017-01-01T00:00:00Z,e9ea9d49-bc94-446e-a741-0d2ce0f13db6,8db78572-ef7d-4821-bca0-258eb63d65d5,447.86,False,-,-
6,7,2017-01-01T00:00:00Z,b2b51b60-5c89-406e-aa1b-248fea94d306,ade81084-ef29-4413-bfda-e6661b314321,387.49,False,-,-
7,8,2017-01-01T00:00:00Z,4dbb5559-8c12-46ef-b1fa-b84d80dd8005,e9f746e7-1fd3-48d0-b7a4-d097aaa22344,370.12,False,-,-
8,9,2017-01-01T00:00:00Z,57232faa-b578-4525-af0a-c5d6ee5e1fb5,a2e25753-56ff-4b3e-b80c-fd3dba6a741c,409.68,False,-,-
9,10,2017-01-01T00:00:00Z,47020de0-bc64-47e9-b16e-c0b6e3d9de7f,99474e78-77cd-4b25-8786-140f2339df60,429.27,Unknown,Unknown,Unknown


In [17]:
accounts.head()

,acct_id,initial_deposit,cust_forname,cust_surname,cust_gender,cust_birthdate
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,92221.09,Terry,Knight,m,1971-11-26
1,23b8c1e9-3924-46de-beb1-3b9046685257,87897.72,Adrian,Walsh,m,1954-01-27
2,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,71028.58,Lynne,Taylor,f,1965-04-02
3,972a8469-1641-4f82-8b9d-2434e465e150,62945.84,Bryan,Hill,m,1958-11-06
4,17fc695a-07a0-4a6e-8822-e8f36c031199,75563.74,Clifford,Jones,m,1976-03-21


In [0]:
#fanout = transactions.groupby('src_acct').agg({'amount': np.mean, 'src_acct': np.size, 'rcv_acct': 'nunique'})
fanout = transactions.groupby('src_acct', 'recv_acct').agg({'amount': np.mean}).nlargest(10, )
fanin = transactions.groupby('rcv_acct').agg({'amount': np.mean, 'rcv_acct': np.size, 'src_acct': 'nunique'})

In [0]:
fanout['total'] = fanout['amount'] * fanout[]

In [0]:
print(fanin.head())
fanout.head()

                                          amount  rcv_acct  src_acct
rcv_acct                                                            
0034801d-a4f8-4cc3-b6ad-9d6e4f7bdd57  897.752500        20        12
00a2cb5b-0fb7-412b-8beb-1eb79862efc8  840.283617        47        15
01d61262-1637-4c52-a4f7-e95a2f1f6882  956.441646        79        50
0264b540-e0a0-4afb-9984-b8cd6c477997  887.082400        50        12
03a5f48f-2383-4522-9d54-af450a95e804  884.554667        45        22


,amount,src_acct,rcv_acct
src_acct,,,
03a5f48f-2383-4522-9d54-af450a95e804,650.678148,54,45
0b5db882-4032-4d24-bf51-56dc9e3fc4a8,1031.592090,67,47
0b73ed81-ad81-4235-a655-3594fd5e76ae,999.110000,56,43
16160e05-7a25-4f35-b667-febde4020a98,1116.080000,52,40
17153d31-1675-41ae-8580-7e801a4ff9f3,1044.930000,49,36


In [0]:

g = pd.merge(transactions, fanout.set_index('src_acct'), on='src_acct', right_index=True)

In [0]:
transactions.groupby('src_acct')

In [0]:
field = 'alert_id'
transactions.groupby('src_acct').agg({field:'nunique'}).nlargest(11, columns=field)


,alert_id
src_acct,
5c9c5809-d27d-4d3f-bbc4-0886de137a41,7
db749698-b608-40b4-9679-9e027312660c,6
4563fca4-383a-408f-bf78-a0255b2cd8e7,5
d9cb07a0-8405-4435-9d4c-b81348aa7a42,5
ea151653-ab77-4bbb-9a52-8fdcdf168f27,5
f447a08f-310f-4e24-97a8-a610cf115457,5
10b4ce98-b433-4d5d-9344-7b3f1e005c06,4
153ff9a5-d48d-4d8b-b602-939d97a8e9ea,4
21738e1e-c0a9-48ae-b507-1d719e1a156c,4


In [0]:
txBySourceNode = transactions.groupby('src_acct')
txByTargetNode = transactions.groupby('rcv_acct')
inDegree = txBySourceNode.size()
outDegree = txByTargetNode.size()
inVolume = txBySourceNode['amount'].sum().rename(columns={0:''})
outVolume = txByTargetNode['amount'].sum().rename(columns={0:''})
dfs = [nodes, inDegree, outDegree, inVolume, outVolume]
colNames = {0: 'inDegree', 1:'outDegree', 2:'inVolume', 3:'outVolume'}


#create dataframe that is ordered by inVolume
#drop the first 18900 results 
#take each row and divide the inVolume by the max inVolume and ratio of Fraud to Non-Fraud
nodesEnriched = pd.concat(dfs, axis=1, join='inner').rename(columns=colNames)
nodesEnriched = nodesEnriched.sort_values(by=['inVolume'])
nodesEnriched.insert(loc=0, column="row", value=np.arange(len(nodesEnriched)))
nodesEnriched.drop(nodesEnriched[nodesEnriched['row']<18900].index, inplace = True)
nodesEnriched['inVolume'] = nodesEnriched['inVolume']/(70102.48*9.26)

#create dataframe that is ordered by inDegree
#drop the first 18900 results
#take each row and divide the inDegree by the max inDegree and ratio of Fraud to Non-Fraud
inTable = pd.concat(dfs, axis=1, join='inner').rename(columns=colNames)
inTable = inTable.sort_values(by=['inDegree'])
inTable.insert(loc=0, column='row', value=np.arange(len(inTable)))
inTable.drop(inTable[inTable['row']< 18900].index, inplace=True)
inTable['inDegree'] = inTable['inDegree']/(205*3.58)

#create dataframe that is ordered by outDegree
#drop the first 18900 results
#take each row and divide the outDegree by the max outDegree and ratio of Fraud to Non-Fraud
outTable = pd.concat(dfs, axis=1, join='inner').rename(columns=colNames)
outTable = outTable.sort_values(by=['outDegree'])
outTable.insert(loc=0, column='row', value=np.arange(len(outTable)))
outTable.drop(outTable[outTable['row']<18900].index, inplace = True)
outTable['outDegree'] = outTable['outDegree']/(171*2.5)


#find the standard deviation of the txBySourceNode
x=txBySourceNode.std(ddof=0)


#create dataframe that is ordered by outVolume
#drop the first 18900 results 
#take each row and divide different columns by there max values and there ratio of Fraud to Non-Fraud
volumeTable = pd.concat(dfs, axis=1, join='inner').rename(columns=colNames)
volumeTable = volumeTable.sort_values(by=['outVolume'])
volumeTable.insert(loc=0, column="row", value=np.arange(len(volumeTable)))
volumeTable.drop(volumeTable[volumeTable['row']< 18900].index, inplace = True)
volumeTable['inDegree'] = volumeTable['inDegree']/(205*3.58)
volumeTable['outDegree'] = volumeTable['outDegree']/(171*2.5)
volumeTable['inVolume'] = volumeTable['inVolume']/(70102.48*9.26)
volumeTable['outVolume'] = volumeTable['outVolume']/(50080.75*4.57) 

#create dataframe
#drop the first 18900 results
#take each row and divide different columns by there max values and there ratio of Fraud to Non-Fraud
#order the dataframe by the sum of the new values for inDegree, outDegree, inVolume and outVolume
addedTable = pd.concat(dfs, axis=1, join='inner').rename(columns=colNames)
addedTable.insert(loc=0,column='row', value=np.arange(len(addedTable)))
addedTable.drop(addedTable[addedTable['row']< 18900].index, inplace = True)
addedTable['inDegree'] = addedTable['inDegree']/(205*3.58)
addedTable['outDegree'] = addedTable['outDegree']/(171*2.5)
addedTable['inVolume'] = addedTable['inVolume']/(70102.48*9.26)
addedTable['outVolume'] = addedTable['outVolume']/(50080.75*4.57)
addedTable['sum'] = addedTable['inDegree'] + addedTable['outDegree'] + addedTable['inVolume'] + addedTable['outVolume']
addedTable = addedTable.sort_values(by=['sum'])

NameError: ignored

In [0]:

addedTable.tail()
addedTable.describe()


In [0]:
#create a scatterplot of inVolume against outVolume
ax1 = nodesEnriched.plot.scatter(x='inVolume', y='outVolume', c='isFraud', colormap='viridis')

In [0]:
#create scatterplot of row number against inVolume 
#where the row number has been ordered in size of inVolume
ax2 = nodesEnriched.plot.scatter(x='row', y='inVolume', c='isFraud', colormap='viridis')

In [0]:
#create scatterplot of row number against inDegree 
#where the row number has been ordered in size of inDegree
ax3 = inTable.plot.scatter(x='row', y='inDegree', c='isFraud', colormap='viridis')

In [0]:
#create scatterplot of row number against outDegree 
#where the row number has been ordered in size of outDegree
ax4 = outTable.plot.scatter(x='row', y='outDegree', c='isFraud', colormap='viridis')

In [0]:
#create scatterplot of row number against outVolume 
#where the row number has been ordered in size of outVolume
ax5 = volumeTable.plot.scatter(x='row', y='outVolume', c='isFraud', colormap='viridis')

In [0]:
#create scatterplot of row number against sum of the columns 
#where the row number has been ordered in size of the sum
ax6 = addedTable.plot.scatter(x='row', y='sum', c='isFraud', colormap='viridis')

In [0]:
#create scatterplot of row number against sum of the columns 
#where the row number has been ordered in size of the sum
sns.lmplot("row", "sum", data=addedTable, hue="isFraud")


In [0]:
sns.set(rc={'figure.figsize':(30.6,15.9)})

In [0]:
#create scatterplot of row number against sum of the columns 
#where the row number has been ordered in size of the sum
sns.scatterplot("row", "sum", data=addedTable, hue="isFraud", size_order=["T1","T2"])